# Different Index Types in LlamaIndex

LlamaIndex offers multiple index types, each optimized for different use cases. Understanding when to use each type is crucial for building effective RAG systems.

## Learning Objectives

By the end of this notebook, you will:
1. Understand the different index types and their trade-offs
2. Know when to use each index type
3. Be able to combine multiple indexes
4. Implement hybrid retrieval strategies

---

## Overview of Index Types

| Index Type | Description | Best For |
|------------|-------------|----------|
| **VectorStoreIndex** | Semantic similarity search | General Q&A, semantic matching |
| **SummaryIndex** | Sequential traversal of all nodes | Summarization, complete coverage |
| **TreeIndex** | Hierarchical tree structure | Large documents, drilling down |
| **KeywordTableIndex** | Keyword-based retrieval | Exact term matching |
| **KnowledgeGraphIndex** | Graph-based relationships | Entity relationships |

In [ ]:
# Setup
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

from llama_index.core import (
    VectorStoreIndex,
    SummaryIndex,
    TreeIndex,
    KeywordTableIndex,
    SimpleDirectoryReader,
    Settings,
    Document,
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Configure
Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

print("✓ Setup complete!")

In [ ]:
# Load documents
documents = SimpleDirectoryReader("../data/sample_docs").load_data()
print(f"Loaded {len(documents)} documents")

## 1. VectorStoreIndex (Default)

The most common index type. Uses embeddings for semantic similarity search.

**How it works:**
1. Documents are chunked into nodes
2. Each node is converted to an embedding vector
3. Queries are also converted to embeddings
4. Most similar vectors are retrieved

**Best for:**
- Semantic similarity questions
- When exact keywords might not be in the document
- General-purpose Q&A

In [ ]:
# Build VectorStoreIndex
print("Building VectorStoreIndex...")
vector_index = VectorStoreIndex.from_documents(
    documents,
    show_progress=True,
)

# Create query engine
vector_query_engine = vector_index.as_query_engine(
    similarity_top_k=3,
)

print("✓ VectorStoreIndex ready!")

In [ ]:
# Test semantic query - works well with paraphrasing
query = "How do computers learn from data?"  # No exact match, but semantically similar

print(f"Query: {query}")
response = vector_query_engine.query(query)
print(f"\nResponse: {response}")

## 2. SummaryIndex (formerly ListIndex)

Processes ALL nodes sequentially. Guarantees complete coverage but is slower and more expensive.

**How it works:**
1. Documents are chunked into nodes
2. At query time, ALL nodes are sent to the LLM
3. Response is synthesized from all content

**Best for:**
- Summarization tasks
- When you need to consider ALL information
- Small document collections

In [ ]:
# Build SummaryIndex
print("Building SummaryIndex...")
summary_index = SummaryIndex.from_documents(
    documents,
    show_progress=True,
)

# Create query engine
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",  # Good for summarization
)

print("✓ SummaryIndex ready!")

In [ ]:
# Summarization query - SummaryIndex excels at this
query = "Provide a comprehensive summary of all the topics covered in these documents."

print(f"Query: {query}")
print("\n(This may take a moment as it processes all nodes...)\n")

response = summary_query_engine.query(query)
print(f"Response: {response}")

## 3. TreeIndex

Builds a hierarchical tree structure for multi-level summarization and retrieval.

**How it works:**
1. Documents are chunked into leaf nodes
2. Leaf nodes are summarized into parent nodes
3. Process repeats until reaching the root
4. Query traverses the tree to find relevant branches

**Best for:**
- Large documents where you want to drill down
- Hierarchical content (books, manuals)
- When you need both overview and detail

In [ ]:
# Build TreeIndex
print("Building TreeIndex...")
print("(This creates a hierarchical summary structure)\n")

tree_index = TreeIndex.from_documents(
    documents,
    show_progress=True,
    num_children=3,  # Number of children per parent node
)

# Create query engine
tree_query_engine = tree_index.as_query_engine(
    child_branch_factor=2,  # How many branches to explore
)

print("\n✓ TreeIndex ready!")

In [ ]:
# Tree traversal query
query = "What are the main programming concepts discussed?"

print(f"Query: {query}")
response = tree_query_engine.query(query)
print(f"\nResponse: {response}")

## 4. KeywordTableIndex

Uses keyword extraction for retrieval. Good when exact terminology matters.

**How it works:**
1. Keywords are extracted from each node (using LLM)
2. A keyword-to-node mapping table is created
3. Query keywords are extracted and matched
4. Matching nodes are retrieved

**Best for:**
- Technical documentation with specific terms
- When exact keyword matching is important
- Glossary-style lookups

In [ ]:
# Build KeywordTableIndex
print("Building KeywordTableIndex...")
print("(This extracts keywords from each node)\n")

keyword_index = KeywordTableIndex.from_documents(
    documents,
    show_progress=True,
)

# Create query engine
keyword_query_engine = keyword_index.as_query_engine()

print("\n✓ KeywordTableIndex ready!")

In [ ]:
# Keyword-based query - exact terms work best
query = "What is supervised learning?"

print(f"Query: {query}")
response = keyword_query_engine.query(query)
print(f"\nResponse: {response}")

## 5. Comparing Index Types

Let's compare how different indexes respond to the same query:

In [ ]:
# Comparison query
test_query = "What are the applications of artificial intelligence?"

indexes = {
    "VectorStoreIndex": vector_query_engine,
    "SummaryIndex": summary_query_engine,
    "TreeIndex": tree_query_engine,
    "KeywordTableIndex": keyword_query_engine,
}

print(f"Query: {test_query}\n")
print("=" * 70)

for name, engine in indexes.items():
    print(f"\n{name}:")
    print("-" * 70)
    try:
        response = engine.query(test_query)
        print(f"{str(response)[:400]}...")
    except Exception as e:
        print(f"Error: {e}")
    print()

## 6. Combining Multiple Indexes (ComposableGraph)

You can combine multiple indexes for more sophisticated retrieval:

In [ ]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.tools import QueryEngineTool

# Create tools for each index type
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for semantic similarity questions about AI, Python, and general topics. "
                "Use when the question requires understanding concepts or finding related information.",
)

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description="Useful for summarization requests or when you need a comprehensive overview "
                "of all the content. Use for questions like 'summarize' or 'give an overview'.",
)

keyword_tool = QueryEngineTool.from_defaults(
    query_engine=keyword_query_engine,
    description="Useful for looking up specific technical terms or definitions. "
                "Use when the question asks about a specific concept by name.",
)

print("✓ Query tools created!")

In [ ]:
# Create a Router Query Engine that automatically selects the best tool
router_query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        vector_tool,
        summary_tool,
        keyword_tool,
    ],
    verbose=True,  # Show which tool was selected
)

print("✓ Router Query Engine ready!")

In [ ]:
# Test the router with different query types
test_queries = [
    "What is machine learning?",  # Should use keyword or vector
    "Summarize all the content about Python.",  # Should use summary
    "How are neural networks related to deep learning?",  # Should use vector
]

for query in test_queries:
    print(f"\n{'='*70}")
    print(f"Query: {query}")
    print("-" * 70)
    response = router_query_engine.query(query)
    print(f"\nResponse: {str(response)[:300]}...")

## 7. Sub-Question Query Engine

For complex questions that span multiple topics:

In [ ]:
from llama_index.core.query_engine import SubQuestionQueryEngine

# Create sub-question query engine
sub_question_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[
        QueryEngineTool.from_defaults(
            query_engine=vector_query_engine,
            description="Contains information about AI, machine learning, Python programming, and technology.",
        ),
    ],
    verbose=True,
)

print("✓ Sub-Question Query Engine ready!")

In [ ]:
# Complex question that benefits from decomposition
complex_query = "Compare and contrast Python programming with AI development practices."

print(f"Complex Query: {complex_query}")
print("\n(Watch how the query is decomposed into sub-questions...)\n")

response = sub_question_engine.query(complex_query)
print(f"\nFinal Response:\n{response}")

## 8. Index Selection Guidelines

### Decision Matrix

| Scenario | Recommended Index | Why |
|----------|------------------|-----|
| General Q&A | VectorStoreIndex | Semantic matching handles paraphrasing |
| Summarization | SummaryIndex | Considers all content |
| Large documents | TreeIndex | Efficient hierarchical navigation |
| Technical docs | KeywordTableIndex | Exact term matching |
| Multi-topic | Router/SubQuestion | Combines strengths |

### Performance Considerations

| Index | Build Time | Query Time | Cost |
|-------|------------|------------|------|
| VectorStoreIndex | Moderate (embeddings) | Fast | Low |
| SummaryIndex | Fast (no embeddings) | Slow (all nodes) | High |
| TreeIndex | Slow (summarization) | Moderate | Moderate |
| KeywordTableIndex | Moderate (extraction) | Fast | Low |

## 9. Summary

You've learned about different index types in LlamaIndex:

### Key Takeaways

1. **VectorStoreIndex** is the default and works well for most semantic search use cases
2. **SummaryIndex** guarantees complete coverage but is slower and more expensive
3. **TreeIndex** provides hierarchical navigation for large documents
4. **KeywordTableIndex** excels at exact term matching
5. **Router** and **SubQuestion** engines combine multiple indexes intelligently

### Best Practices

1. Start with VectorStoreIndex for most use cases
2. Use SummaryIndex only when complete coverage is essential
3. Consider combining indexes for complex applications
4. Profile performance based on your specific needs

### Next Steps

In the next notebook, we'll explore custom retrievers and reranking strategies.

---

## Exercises

1. **Index comparison**: Run 10 different queries and note which index performs best for each

2. **Custom router**: Create a router with custom selection logic

3. **Hybrid approach**: Combine vector and keyword search results

4. **Performance test**: Measure build time and query time for each index type

In [ ]:
# Exercise space
# Try different index configurations here!